# 1. Perkenalan

```
Program ini dibuat untuk menganalisa penjualan perusahaan selama 1 minggu pertama di Bulan Juli 2024. Hasil visualisasi dari analisa data ini akan dituangkan dalam Tableau pada link berikut: [Tableau]() 

## Business Understanding

SMART FrameWork
* Spesific => 
* Measurable => 
* Achieveble =>
* Relevant =>
* Time Bond => 

Problem Statement

# 2. Import Libraries

In [126]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# 3. Data Loading

In [127]:
df = pd.read_csv("dataset.csv")
df

,DATE,AIRLINES,ORIGIN,DESTINATION,FLIGHT_NUMB,COLLY,WEIGHT,CUSTOMER,CUST_ID,SERVICE,COST_A,COST_B,COST_C,PRICE
0,01/Jul/2024,CITILINK,JAKARTA,MEDAN,QG-0916,5,"72,00",PT DINDA SETIA KIRANA,DSK,PTP,9324,4343,2809,11300
1,01/Jul/2024,CITILINK,JAKARTA,BALIKPAPAN,QG-0436,1,"15,00",PT FAJRI ABANGDA PRATAMA,FAP,PTP,15984,4343,1258,31200
2,01/Jul/2024,CITILINK,JAKARTA,MAKASSAR,QG-0426,6,"77,00",PT FAJRI ABANGDA PRATAMA,FAP,PTP,13653,4343,1594,34800
3,01/Jul/2024,CITILINK,JAKARTA,BATAM,QG-0968,6,"95,00",PT DINDA SETIA KIRANA,DSK,PTP,10545,4343,2280,11050
4,01/Jul/2024,CITILINK,JAKARTA,BALIKPAPAN,QG-0424,7,"100,00",PT DINDA SETIA KIRANA,DSK,PTP,15984,4343,1258,22100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28658,07/Jul/2025,FREIGHTER,JAKARTA,TARAKAN,ECA245,2,"10,00",PT BAYA SKENA MENYALA,BSM,DTD,35964,3888,0,42870
28659,07/Jul/2024,FREIGHTER,JAKARTA,TARAKAN,ECA245,5,"51,00",PT DINDA SETIA KIRANA,DSK,PTP,35964,3888,0,38200
28660,07/Jul/2024,FREIGHTER,JAKARTA,TARAKAN,ECA245,11,"123,00",PT DINDA SETIA KIRANA,DSK,PTP,35964,3888,0,38200
28661,07/Jul/2024,FREIGHTER,JAKARTA,TARAKAN,ECA245,10,"127,00",PT DINDA SETIA KIRANA,DSK,PTP,35964,3888,0,38200


Data terdisi dari 14 kolom dan 28.663 baris. Berikut ini penjelasan informasi dari setiap kolom:
* DATE          : Tanggal transaksi 
* AIRLINES      : Maskapai yang digunakan untuk pengiriman cargo 
* ORIGIN        : Kota asal pengiriman 
* DESTINATION   : Kota tujuan pengiriman
* FLIGHT_NUMB   : Kode penerbangan berdasarkan maskapai dan rute
* COLLY         : Jumlah karung pengiriman 
* WEIGHT        : Total berat pengiriman dari semua jumlah karungan 
* CUSTOMER      : Nama Customer 
* CUST_ID       : Kode unik Customer 
* SERVICE       : Jenis service pengiriman yang digunakan 
* COST_A        : Komponen cost 1 
* COST_B        : Komponen cost 2 
* COST_C        : Komponen cost 3 
* PRICE         : Harga jual per Kg 

Selanjutnya akan dilakukan pengecekan terhadap jenis data dari masing-masing kolom dan keberadaan missing value.

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28663 entries, 0 to 28662
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   DATE         28663 non-null  object
 1   AIRLINES     28663 non-null  object
 2   ORIGIN       28663 non-null  object
 3   DESTINATION  28663 non-null  object
 4   FLIGHT_NUMB  28663 non-null  object
 5   COLLY        28663 non-null  int64 
 6    WEIGHT      28663 non-null  object
 7   CUSTOMER     28654 non-null  object
 8   CUST_ID      28663 non-null  object
 9   SERVICE      28663 non-null  object
 10  COST_A       28663 non-null  int64 
 11  COST_B       28663 non-null  int64 
 12  COST_C       28663 non-null  int64 
 13  PRICE        28663 non-null  int64 
dtypes: int64(5), object(9)
memory usage: 3.1+ MB


Dapat dilihat bahwa data memiliki beberapa 9 missing value pada kolom CUSTOMER dan terdapat kolom DATE dan kolom WEIGHT yang memiliki jenis data yang tidak sesuai

In [129]:
df['CUSTOMER'].unique()

array(['PT DINDA SETIA KIRANA', 'PT FAJRI ABANGDA PRATAMA', nan,
       'PT ADANI SUBUR MAKMUR', 'PT BAYA SKENA MENYALA',
       'PT HABIB REZA ALKARIM', 'PT PRIMA CAHAYA MEGA'], dtype=object)

In [130]:
df['CUSTOMER'].mode()

0    PT PRIMA CAHAYA MEGA
Name: CUSTOMER, dtype: object

# 4. Data Cleaning

## Penghapusan White Space Pada Nama Kolom dan Value Kolom WEIGHT

In [131]:
df.columns = df.columns.str.strip()

In [132]:
df.columns

Index(['DATE', 'AIRLINES', 'ORIGIN', 'DESTINATION', 'FLIGHT_NUMB', 'COLLY',
       'WEIGHT', 'CUSTOMER', 'CUST_ID', 'SERVICE', 'COST_A', 'COST_B',
       'COST_C', 'PRICE'],
      dtype='object')

In [133]:
df['WEIGHT'].unique()

array(['  72,00 ', '  15,00 ', '  77,00 ', '  95,00 ', '  100,00 ',
       '  17,00 ', '  40,00 ', '  157,00 ', '  89,00 ', '  10,00 ',
       '  18,00 ', '  21,00 ', '  171,00 ', '  147,00 ', '  66,00 ',
       '  334,00 ', '  23,00 ', '  90,00 ', '  137,00 ', '  217,00 ',
       '  323,00 ', '  85,00 ', '  34,00 ', '  55,00 ', '  12,00 ',
       '  78,00 ', '  69,00 ', '  141,00 ', '  170,00 ', '  36,00 ',
       '  50,00 ', '  27,00 ', '  11,00 ', '  56,00 ', '  58,00 ',
       '  47,00 ', '  28,00 ', '  63,00 ', '  62,00 ', '  98,00 ',
       '  200,00 ', '  81,00 ', '  19,00 ', '  26,00 ', '  24,00 ',
       '  61,00 ', '  31,00 ', '  173,00 ', '  74,00 ', '  53,00 ',
       '  142,00 ', '  13,00 ', '  42,00 ', '  29,00 ', '  73,00 ',
       '  38,00 ', '  60,00 ', '  14,00 ', '  112,00 ', '  80,00 ',
       '  20,00 ', '  43,00 ', '  37,00 ', '  218,00 ', '  35,00 ',
       '  190,00 ', '  114,00 ', '  160,00 ', '  59,00 ', '  232,00 ',
       '  520,00 ', '  252,00 ', '  254,00 

In [134]:
df['WEIGHT'] = df['WEIGHT'].str.replace(" ", "").str.replace('.', '').str.replace(',', '.')
df['WEIGHT'].unique()

array(['72.00', '15.00', '77.00', '95.00', '100.00', '17.00', '40.00',
       '157.00', '89.00', '10.00', '18.00', '21.00', '171.00', '147.00',
       '66.00', '334.00', '23.00', '90.00', '137.00', '217.00', '323.00',
       '85.00', '34.00', '55.00', '12.00', '78.00', '69.00', '141.00',
       '170.00', '36.00', '50.00', '27.00', '11.00', '56.00', '58.00',
       '47.00', '28.00', '63.00', '62.00', '98.00', '200.00', '81.00',
       '19.00', '26.00', '24.00', '61.00', '31.00', '173.00', '74.00',
       '53.00', '142.00', '13.00', '42.00', '29.00', '73.00', '38.00',
       '60.00', '14.00', '112.00', '80.00', '20.00', '43.00', '37.00',
       '218.00', '35.00', '190.00', '114.00', '160.00', '59.00', '232.00',
       '520.00', '252.00', '254.00', '464.00', '120.00', '296.00',
       '71.00', '298.00', '82.00', '384.00', '93.00', '294.00', '345.00',
       '46.00', '135.00', '584.00', '119.00', '340.00', '196.00',
       '293.00', '462.00', '41.00', '359.00', '447.00', '152.00',
       '

## Ubah Tipe Data

In [135]:
df['WEIGHT'] = df['WEIGHT'].astype(float)
df['DATE'] = pd.to_datetime(df['DATE'])
df[['WEIGHT', 'DATE']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28663 entries, 0 to 28662
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   WEIGHT  28663 non-null  float64       
 1   DATE    28663 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1)
memory usage: 448.0 KB


In [136]:
df['DATE'] = pd.to_datetime(df['DATE'])

## Handling Missing Value

Handling missing value yang terdapat pada kolom CUSTOMER akan dilakukan dengan cara melihat pola yang terdapat pada kolom lain, dalam hal ini adalah kolom CUST_ID

In [137]:
df[df['CUSTOMER'].isna()]

,DATE,AIRLINES,ORIGIN,DESTINATION,FLIGHT_NUMB,COLLY,WEIGHT,CUSTOMER,CUST_ID,SERVICE,COST_A,COST_B,COST_C,PRICE
22,2024-07-01,CITILINK,JAKARTA,MEDAN,QG-0978,9,90.0,NaN,FAP,PTP,9324,4343,2809,25200
65,2024-07-01,CITILINK,JAKARTA,DENPASAR,QG-0684,1,11.0,NaN,DSK,PTP,8103,4343,2200,9025
28193,2024-07-04,FREIGHTER,JAKARTA,TERNATE,RA-2024,5,44.0,NaN,BSM,DTD,43290,4010,0,51690
28194,2024-07-04,FREIGHTER,JAKARTA,PALU,RA-2024,16,140.0,NaN,MEG,DTD,36297,5010,0,47795
28195,2024-07-04,FREIGHTER,JAKARTA,TERNATE,RA-2024,5,55.0,NaN,BSM,DTD,43290,4010,0,51690
28323,2024-07-06,FREIGHTER,JAKARTA,PALU,RA-2024,2,39.0,NaN,MEG,DTD,36297,5010,0,47795
28419,2024-07-07,FREIGHTER,JAKARTA,PALU,RA-2024,3,35.0,NaN,DSK,PTP,36297,4010,0,42250
28594,2024-07-05,FREIGHTER,JAKARTA,TARAKAN,ECA245,15,150.0,NaN,DSK,PTP,35964,3888,0,38200
28646,2025-07-07,FREIGHTER,JAKARTA,TARAKAN,ECA245,3,41.0,NaN,BSM,DTD,35964,3888,0,42870


In [138]:
customer_id_target = ['FAP', 'DSK', 'BSM', 'MEG']
seen = set ()

for index, row in df.iterrows():
    if row['CUST_ID'] in customer_id_target and row['CUST_ID'] not in seen:
        print(f"Customer {row['CUST_ID']}: {row['CUSTOMER']}")
        seen.add(row['CUST_ID'])

Customer DSK: PT DINDA SETIA KIRANA
Customer FAP: PT FAJRI ABANGDA PRATAMA
Customer BSM: PT BAYA SKENA MENYALA
Customer MEG: PT PRIMA CAHAYA MEGA


In [139]:
df[(df['FLIGHT_NUMB'] == 'RA-2024') & (df['CUST_ID'] == 'MEG') & (df['COLLY'] == 2)  & (df['WEIGHT'] == 39)]

,DATE,AIRLINES,ORIGIN,DESTINATION,FLIGHT_NUMB,COLLY,WEIGHT,CUSTOMER,CUST_ID,SERVICE,COST_A,COST_B,COST_C,PRICE
28323,2024-07-06,FREIGHTER,JAKARTA,PALU,RA-2024,2,39.0,NaN,MEG,DTD,36297,5010,0,47795


In [140]:
def fill_missing(row):
    if pd.isna(row['CUSTOMER']):
        if (row['FLIGHT_NUMB'] == 'QG-0978') & (row['CUST_ID'] == 'FAP') & (row['COLLY'] == 9):
            return "PT FAJRI ABANGDA PRATAMA"
        if (row['CUST_ID'] == 'DSK'):
            if ((row['FLIGHT_NUMB'] == 'QG-0684') & (row['COLLY'] == 1) & (row['WEIGHT'] == 11)) or \
                ((row['FLIGHT_NUMB'] == 'RA-2024') & (row['COLLY'] == 3) & (row['WEIGHT'] == 35)) or \
                ((row['FLIGHT_NUMB'] == 'ECA245') & (row['COLLY'] == 15) & (row['WEIGHT'] == 150)): 
                    return "PT DINDA SETIA KIRANA"
        elif (row['CUST_ID'] == 'BSM'):
            if ((row['FLIGHT_NUMB'] == 'RA-2024') & (row['COLLY'] == 5) & (row['WEIGHT'] == 44)) or \
                ((row['FLIGHT_NUMB'] == 'RA-2024') & (row['COLLY'] == 5) & (row['WEIGHT'] == 55)) or \
                ((row['FLIGHT_NUMB'] == 'ECA245') & (row['COLLY'] == 3) & (row['WEIGHT'] == 41)):
                    return 'PT BAYA SKENA MENYALA'
        elif (row['CUST_ID'] == 'MEG'):
            if ((row['FLIGHT_NUMB'] == 'RA-2024') & (row['COLLY'] == 16) & (row['WEIGHT'] == 140)) or \
                ((row['FLIGHT_NUMB'] == 'RA-2024') & (row['COLLY'] == 2) & (row['WEIGHT'] == 39)):
                    return "PT PRIMA CAHAYA MEGA"
    return row['CUSTOMER']

#apply ke kolom 'CUSTOMER'
df['CUSTOMER'] = df.apply(fill_missing, axis=1)

print(df)

            DATE   AIRLINES   ORIGIN DESTINATION FLIGHT_NUMB  COLLY  WEIGHT  \
0     2024-07-01   CITILINK  JAKARTA       MEDAN     QG-0916      5    72.0   
1     2024-07-01   CITILINK  JAKARTA  BALIKPAPAN     QG-0436      1    15.0   
2     2024-07-01   CITILINK  JAKARTA    MAKASSAR     QG-0426      6    77.0   
3     2024-07-01   CITILINK  JAKARTA       BATAM     QG-0968      6    95.0   
4     2024-07-01   CITILINK  JAKARTA  BALIKPAPAN     QG-0424      7   100.0   
...          ...        ...      ...         ...         ...    ...     ...   
28658 2025-07-07  FREIGHTER  JAKARTA     TARAKAN      ECA245      2    10.0   
28659 2024-07-07  FREIGHTER  JAKARTA     TARAKAN      ECA245      5    51.0   
28660 2024-07-07  FREIGHTER  JAKARTA     TARAKAN      ECA245     11   123.0   
28661 2024-07-07  FREIGHTER  JAKARTA     TARAKAN      ECA245     10   127.0   
28662 2024-07-07  FREIGHTER  JAKARTA     TARAKAN      ECA245      8   111.0   

                       CUSTOMER CUST_ID SERVICE  CO

## Penghapusan Baris Duplikat

In [141]:
df_clean = df.drop_duplicates()
df_clean

,DATE,AIRLINES,ORIGIN,DESTINATION,FLIGHT_NUMB,COLLY,WEIGHT,CUSTOMER,CUST_ID,SERVICE,COST_A,COST_B,COST_C,PRICE
0,2024-07-01,CITILINK,JAKARTA,MEDAN,QG-0916,5,72.0,PT DINDA SETIA KIRANA,DSK,PTP,9324,4343,2809,11300
1,2024-07-01,CITILINK,JAKARTA,BALIKPAPAN,QG-0436,1,15.0,PT FAJRI ABANGDA PRATAMA,FAP,PTP,15984,4343,1258,31200
2,2024-07-01,CITILINK,JAKARTA,MAKASSAR,QG-0426,6,77.0,PT FAJRI ABANGDA PRATAMA,FAP,PTP,13653,4343,1594,34800
3,2024-07-01,CITILINK,JAKARTA,BATAM,QG-0968,6,95.0,PT DINDA SETIA KIRANA,DSK,PTP,10545,4343,2280,11050
4,2024-07-01,CITILINK,JAKARTA,BALIKPAPAN,QG-0424,7,100.0,PT DINDA SETIA KIRANA,DSK,PTP,15984,4343,1258,22100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28658,2025-07-07,FREIGHTER,JAKARTA,TARAKAN,ECA245,2,10.0,PT BAYA SKENA MENYALA,BSM,DTD,35964,3888,0,42870
28659,2024-07-07,FREIGHTER,JAKARTA,TARAKAN,ECA245,5,51.0,PT DINDA SETIA KIRANA,DSK,PTP,35964,3888,0,38200
28660,2024-07-07,FREIGHTER,JAKARTA,TARAKAN,ECA245,11,123.0,PT DINDA SETIA KIRANA,DSK,PTP,35964,3888,0,38200
28661,2024-07-07,FREIGHTER,JAKARTA,TARAKAN,ECA245,10,127.0,PT DINDA SETIA KIRANA,DSK,PTP,35964,3888,0,38200


Selanjutnya, akan dilakukan pengecekan nilai perhitungan statistik

In [144]:
df_clean.describe().T

,count,mean,min,25%,50%,75%,max,std
DATE,25093,2024-07-06 14:05:56.083369984,2024-07-01 00:00:00,2024-07-02 00:00:00,2024-07-04 00:00:00,2024-07-06 00:00:00,2025-07-07 00:00:00,NaN
COLLY,25093.0,9.246921,1.0,3.0,8.0,15.0,217.0,8.042857
WEIGHT,25093.0,204.643601,1.0,32.0,74.0,145.0,1435628.0,11572.514583
COST_A,25093.0,20370.813334,3330.0,10545.0,14153.0,27972.0,96348.0,13331.814282
COST_B,25093.0,3505.42737,1144.0,3060.0,3282.0,4343.0,7596.0,818.827061
COST_C,25093.0,1686.296338,0.0,1343.0,1594.0,2280.0,4085.0,858.788335
PRICE,25093.0,28322.315785,5320.0,19690.0,23770.0,37000.0,109700.0,12900.925953
